In [1]:
import pandas as pd
import numpy as np
import os

columns = ['date','num_series', 'voltage', 'current', 'v_err', 'i_err']
data = pd.read_csv('../files/cellsense/2019-12-01.csv', encoding='cp1251',sep=';', names=columns)


In [2]:
def change_date(x):
    ### Получаем дату из формата [date time]
    date = x.split(' ')[0]
    return date

def to_float(x):
    ### Перевод напряжения из типа строки в float
    if isinstance(x,str):
        x = x.replace(',','.')
        return float(x)
    else: return x

In [3]:
def get_pivot(df):
    ### Формирование общей таблицы - среднее напряжение по всем сериям за месяц
    df = df[df.num_series.notnull()]
    _df_res = pd.DataFrame({'date':0, 'voltage': 0, 'num_series':0}, index=[0])
    
    for item in range(1, 33):
        _df_item = create_df(df,item)
        _df_res = pd.concat([_df_res, _df_item], ignore_index=True)
    _df_res.drop([0], inplace=True)
    return _df_res

In [4]:
def create_df(df, num_series):
    ### Формирование сводной таблицы - среднее напряжение на серии за месяц
    ### В базе данных номер серии и ванны в формате [num_series * 1000 + num_bath]
    num = num_series * 1000 
    _df = df[(df['num_series'] >= num) & (df['num_series'] < (num + 1000))] 

    _df['date'] = _df['date'].apply(change_date)
    _df['voltage'] = _df['voltage'].apply(to_float)

    _df = _df.pivot_table(index='date', values='voltage', aggfunc=np.mean)
    _df = _df.reset_index()
    _df['num_series'] = num_series
    return _df
    

In [5]:
pivot = get_pivot(data)
res = pivot.pivot_table(index='date', values='voltage', columns='num_series', aggfunc=np.mean)
res.to_excel('../files/cellsense/12.xlsx')